In [ ]:
import sys
import os
import re
sys.path.append(os.path.join(r"C:\Users\komputer\srodowiska\ai_writer\Lib\site-packages"))
from bs4 import BeautifulSoup
from selenium.webdriver.common.by import By
from selenium import webdriver
from selenium.webdriver.common.by import By
import time
lista = []

def stablize_connection():
    driver = webdriver.Edge()
    driver.maximize_window()
    return driver


def connect_with_author_site_and_return_song_data(author,page_num):
    driver = stablize_connection()
    driver.get(f'https://www.tekstowo.pl/piosenki_artysty,{author},strona,{page_num}.html')
    elems = driver.find_elements(By.CLASS_NAME,"flex-group")
    elements_list =[elem for elem in elems]
    return elements_list

def cleaning_hrefs(author, elements):
    link_list = []
    final_link_list = []
    if elements != []:
        for elem in elements:
            try:
                new_link = search_for_href(elem)
                link_list.append(new_link)
            except:
                print('failed',link_number)

        href_list = remove_unnecessary_hrefs(link_list,author)
        for href in href_list:
            if href[0]!="/":
                correct_link = "/" + href
                final_link_list.append(correct_link)
            else:
                final_link_list.append(href)
    
    return final_link_list

def search_for_href(element):
    soup = BeautifulSoup(element.get_attribute('innerHTML'),'html.parser')
    a_element = soup.find('a')
    # Extract the href attribute
    href_attribute = a_element.get('href')
    return href_attribute

def remove_unnecessary_hrefs(link_list,author):
    for link in link_list:
        if (f'piosenka,{author}' not in link):
            link_list.remove(link)
    
    link_list = [link for link in link_list]
    clean_list= []
    for link in link_list:
        if link[1]=='p':
            clean_list.append(link)
    return clean_list

def clean_links_for_1_page(author,page_num):
    elements_list = connect_with_author_site_and_return_song_data(author,page_num)
    final_link_list = cleaning_hrefs(author,elements_list)
    return final_link_list

def one_song_text(all_textes,link):
    driver = stablize_connection()
    driver.get(f'https://www.tekstowo.pl{link}')
    driver.implicitly_wait(2)
    div = driver.find_element(By.CLASS_NAME,'inner-text')
    text = div.get_attribute('innerHTML')
    result = re.sub("[\(\[].*?[\)\]]", "", text).replace('<br>'," ").replace("\\",'')
    all_textes.append(result)
    
    
import re
def get_all_authors_textes(author,desired_pages_num):
    all_textes_for_artist = []
    for i in range(1,desired_pages_num+1):
        try:
            list_of_links = clean_links_for_1_page(author, i)
            for href in list_of_links:
                one_song_text(all_textes_for_artist,href)
        except:
            if list_of_links==[]:
                print('nothing on this page')
                break
            else:
                print("records are as web_elements, different problem",i)
    if len(all_textes_for_artist)>0:
        track_list = []
        for text in all_textes_for_artist:
            clean_text = re.sub("[\(\[].*?[\)\]]", "", text)  
            track_list.append(clean_text)
        return track_list     
    else:
        print('no textes for you',i," ",author)
        return []
    

def insert_to_db(track_list,author):
    import psycopg2
    conn = psycopg2.connect("dbname=your-db-name user=your-db-username password=your-db-pass")
    cur = conn.cursor()
    cur.execute(f"CREATE TABLE IF NOT EXISTS {your-table} (id serial PRIMARY KEY, text varchar, author varchar);")
    lista_trakow = [track for track in track_list]
    # Execute a command: this creates a new table
    for track in lista_trakow:
        cur.execute(f"INSERT INTO {your} (text,author) VALUES (%s,%s)",(track,author))

# Make the changes to the database persistent
    conn.commit()
    cur.close()
    conn.close()

    
full_list = [['Drake', 1],
 ['Eminem', 2],
 ['Kanye West', 3],
 ['Juice WRLD', 4],
 ['XXXTENTACION', 5],
 ['travis_scott_', 6],
 ['Kendrick Lamar', 7],
 ['Future', 8],
 ['J. Cole', 9],
 ['Nicki Minaj', 10],
 ['Lil Uzi Vert', 11],
 ['21 Savage', 12],
 ['Lil Baby', 13],
 ['Lil Wayne', 14],
 ['Wiz Khalifa', 15],
 ['Young Thug', 16],
 ['Pop Smoke', 17],
 ['Cardi B', 18],
 ['Mac Miller', 19],
 ['DaBaby', 20],
 ['migos_', 22],
 ['YoungBoy Never Broke Again', 23],
 ['Tyga', 24],
 ['Gunna', 25],
 ['tyler__the_creator_', 26],
 ['Kodak Black', 27],
 ['Polo G', 28],
 ['Trippie Redd', 29],
 ['G-Eazy', 30],
 ['a_ap_rocky', 31],
 ['A Boogie Wit da Hoodie', 32],
 ['Logic', 33],
 ['Jay-Z', 34],
 ['DJ Khaled', 35],
 ['Swae Lee', 36],
 ['Big Sean', 37],
 ['50 Cent', 38],
 ['Kid Cudi', 39],
 ['Macklemore', 40],
 ['2Pac', 41],
 ['Childish Gambino', 42],
 ['Roddy Ricch', 43],
 ['playboi_carti_.', 44],
 ['Russ', 45],
 ['Megan Thee Stallion', 46],
 ['Jack Harlow', 47],
 ['Tory Lanez', 48],
 ['Lil Durk', 49],
 ['french_montana_', 50]]

author_names = [sublist[0].replace('.','_').replace(' ','_').replace('-','_').replace(',','_') for sublist in full_list]
author_names

for author in author_names:
    try:
        track_list = get_all_authors_textes(author=author,desired_pages_num=10)
        if len(track_list)>0:
            insert_to_db(track_list=track_list,author=author)
    except:
        print('Not known error',author)